### Description
Author: T. Majidzadeh

Date Created: March 28, 2025

Date Updated: April 4, 2025

Purpose: Prepare data for Zillow rent indices, 2015-2022, treatment in Dec 2017. Prototype version assumes "affected" metros are those with at least 35% post-merger penetration rate and at least 10% share gain from the merger. Uses ARIMA models to forecast ZORI values in but-for world without COVID in 2020-2022.

#### Setup

In [3]:
import pandas as pd
import numpy as np
import datetime
from statsmodels.tsa.arima.model import ARIMA
import warnings
import json
import os
import re
import altair as alt

In [4]:
paths = {
    "zillow_raw": "..\\data\\zillow_data_raw\\",
    "zillow_reg": "..\\data\\zillow_reg_data\\"
}

In [5]:
zillow_reg = pd.read_pickle(paths['zillow_reg']+'zillow_data_reg_20250404.pkl')

In [6]:
# Base version assumes "Affected" metros are those with at least 35% post-merger penetration rate and at least 10% share gain from the merger.

affected_msas = {
    "Atlanta, GA" : 1,
    "Dallas, TX" : 1,
    "Phoenix, AZ" : 1,
    "Denver, CO" : 1,
    "Tampa, FL" : 1,
    "Washington, DC" : 1,
    
    "Houston, TX" : 0,
    "Riverside, CA" : 0,
    "Las Vegas, NV" : 0,
    "Seattle, WA" : 0,
    "Philadelphia, PA" : 0,
    "Boston, MA" : 0,
    "Minneapolis, MN" : 0,
    "San Diego, CA" : 0,
    "Miami, FL" : 0,
    "San Francisco, CA" : 0,
    "Chicago, IL" : 0,
    "Detroit, MI" : 0,
    "Los Angeles, CA" : 0,
    "New York, NY" : 0
}

In [7]:
first_ym = datetime.datetime.strptime('2015-01-01', '%Y-%m-%d').date()
treat_ym = datetime.datetime.strptime('2017-12-01', '%Y-%m-%d').date()
covid_ym = datetime.datetime.strptime('2020-01-01', '%Y-%m-%d').date()
end_ym = datetime.datetime.strptime('2022-12-31', '%Y-%m-%d').date()
zillow_reg_pre = zillow_reg[zillow_reg['Year-Month'] < covid_ym]
lag_months = 1

def months_since_start(date, start_date):
    return (date.year - start_date.year) * 12 + (date.month - start_date.month)

def lag_term(df, num_months=1):
    sorted_df = df.sort_values(by=['RegionName', 'Year-Month'])
    shifted = sorted_df.groupby('RegionName')['ZORI'].shift(num_months)
    return shifted

def get_time_series(city, df, start_date, end_date):
    '''
    Given a city name, panel data, start date and end date, extract a time series for the ARIMA model.
    Inputs:
        city: A city name, in string format.
        df: A panel dataset which includes at least ZORI values by city and month, 
            in Pandas DataFrame format.
        start_date, end_date: The start and end dates of the time series, in either string
            or datetime.date format (function checks), expressed %Y-%m-%d, inclusive.
    Exports:
        time_series_out: A time series for City from Start Date to End Date.
    '''
    if not isinstance(start_date, datetime.date):
        start_date = datetime.datetime.strptime(start_date, '%Y-%m-%d').date()
    if not isinstance(end_date, datetime.date):
        end_date = datetime.datetime.strptime(end_date, '%Y-%m-%d').date()

    time_series_out = df[
        (df['RegionName'] == city) & 
        (df['Year-Month'] >= start_date) & 
        (df['Year-Month'] <= end_date)]['ZORI']
    return time_series_out

def train_and_forecast_arima(
    arima_params, 
    city, 
    df, 
    train_start_date, 
    train_end_date,
    forecast_months=24
    ):
    '''
    Given a set of ARIMA parameters and a dataframe, create an ARIMA model and score it.
    Inputs:
        arima_params: ARIMA parameters, tuple format ((p, d, q), (P, D, Q, S)).
        city: Which city to fit the model, string format.
        df: A DataFrame with the ZORI values for each city over time.
        train_start_date, train_end_date: Start and end date for training data.
        test_start_date, test_end_date: Start and end date for test data.
            Coerce dates to datetime.date format if they are strings.
        score_func: Function to score.
    Outputs: 
        score: The output of the score_func when comparing predicted values to test data.
    '''
    assert isinstance(arima_params, tuple), "Model params should be a tuple."
    assert isinstance(arima_params[0], tuple), "ARIMA params should be a tuple (p, d, q)"
    assert isinstance(arima_params[1], tuple), "SARIMA params should be a tuple (P, D, Q, S)"
    assert (len(arima_params[0]) == 3) & (len(arima_params[1]) == 4), "Model should be a tuple ((p, d, q), (P, D, Q, S))"
    assert isinstance(city, str), "City should be a string"
    if not isinstance(train_start_date, datetime.date):
        train_start_date = datetime.datetime.strptime(train_start_date, '%Y-%m-%d').date()
    if not isinstance(train_end_date, datetime.date):
        train_end_date = datetime.datetime.strptime(train_end_date, '%Y-%m-%d').date()

    train_series = get_time_series(
            city, df, train_start_date, train_end_date
        )
    
    arima_model = ARIMA(
        train_series, 
        order=arima_params[0], 
        seasonal_order=arima_params[1],
        enforce_stationarity=False,
        enforce_invertibility=False
    )
    arima_model_fit = arima_model.fit()
    forecast = arima_model_fit.forecast(steps=forecast_months)
    return forecast

def build_city_forecast_df(city, post_covid_forecast, forecast_dates, pre_covid_df=zillow_reg_pre, affected_msas=affected_msas):
    '''
    Builds a DataFrame in the same format as the Zillow reg data from a post-COVID ARIMA forecast.
    Inputs:
        city: The string name of the city.
        pre_covid_df: A DataFrame with pre-COVID data.
        post_covid_forecast: ARIMA time series forecasts for a city's ZORI counterfactual without COVID.
        forecast_dates: The date indices for the counterfactual forecast.
        affected_msas: Dictionary marking the affected MSAs based on RealPage's penetration rates.
    Outputs:
        post_covid_df: A DataFrame with the above information formatted to match the zillow_reg dataset.
    '''
    zori = post_covid_forecast
    pre_covid_city = pre_covid_df[pre_covid_df['RegionName'] == city]
    state = pre_covid_city[['StateName']].iloc[0,0]
    affected_city = affected_msas[city]
    year, month = [x.year for x in forecast_dates], [x.month for x in forecast_dates]
    time_trend = [months_since_start(x, first_ym) for x in forecast_dates]
    post_covid_df = pd.DataFrame(data={
        'RegionName': city,
        'StateName': state,
        'Year-Month': forecast_dates,
        'ZORI': post_covid_forecast,
        'AffectedCity': affected_city,
        'AffectedTime': 1, # All forecasts are post-merger.
        'Year': year,
        'Month': month,
        'TimeTrend': time_trend
    })
    return post_covid_df

#### Fit ARIMA models and forecast for each city

In [9]:
forecasts = {}
warnings.filterwarnings('ignore')
for city in zillow_reg['RegionName'].unique():
    if city == 'Chicago, IL':
        spec = ((1, 1, 1), (0, 1, 1, 12)) # Chicago over-fits for this time period and >1 autocorrelation term.
    else:
        spec = ((3, 1, 1), (0, 1, 1, 12))
    forecasts[city] = train_and_forecast_arima(
        arima_params = spec,
        city = city,
        df = zillow_reg_pre,
        train_start_date = treat_ym,
        train_end_date = covid_ym,
        forecast_months = 36
    )

#### Use forecasts to append synthetic counterfactual data

In [11]:
forecast_dates = [x.date() for x in pd.date_range(covid_ym, end_ym,  freq='M').tolist()]

In [12]:
zillow_reg_pre.drop(columns='ZORI-Lag1', inplace=True)

In [13]:
dfs = [zillow_reg_pre]
for city in zillow_reg['RegionName'].unique():
    dfs += [build_city_forecast_df(city=city, post_covid_forecast=forecasts[city], forecast_dates=forecast_dates)]
zillow_reg_with_forecast = pd.concat(dfs).reset_index(drop=True)

In [14]:
zillow_reg_with_forecast['ZORI-Lag1'] = lag_term(zillow_reg_with_forecast, lag_months)

In [15]:
zillow_reg.rename(columns={'ZORI':'ZORIOrig'}, inplace=True)
zillow_reg_with_forecast = pd.merge(
        zillow_reg_with_forecast,
        zillow_reg[['RegionName', 'Year-Month', 'ZORIOrig']],
        on=['RegionName', 'Year-Month']
)

In [16]:
zillow_reg_with_forecast['Year-Month'] = pd.to_datetime(zillow_reg_with_forecast['Year-Month'])
alt.Chart(zillow_reg_with_forecast).mark_line().encode(
    x='TimeTrend:Q',
    y='ZORI:Q',
    color='RegionName',
    tooltip = ['RegionName', 'TimeTrend', 'AffectedCity', 'AffectedTime']
).interactive()

alt.Chart(...)

In [17]:
alt.Chart(zillow_reg_with_forecast).mark_line().encode(
    x='Year-Month',
    y='ZORIOrig:Q',
    color='RegionName',
    tooltip = ['RegionName', 'TimeTrend', 'AffectedCity', 'AffectedTime']
).interactive()

alt.Chart(...)

#### Export

In [19]:
zillow_reg_with_forecast.to_csv(paths['zillow_reg']+'zillow_data_reg_counterfactual_20250404.csv')
zillow_reg_with_forecast.to_pickle(paths['zillow_reg']+'zillow_data_reg_counterfactual_20250404.pkl')